In [8]:
# Adds SCP logic to send array_config_X.dat files before job starts
# Suppresses known_hosts warning without touching stderr

import subprocess
import os
import time
import threading
from datetime import timedelta

# -------- CONFIG SECTION -------- #
remote_configs = {
    "cadencea12@172.16.121.82": {"password": "caduser@123", "subdir": [ "array_config_1",  "array_config_2" ]},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "subdir": ["array_config_5", "array_config_6"]},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "subdir": ["array_config_7", "array_config_8"]},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "subdir": ["array_config_9", "array_config_10"]},
    "cadencea15@172.16.121.6": {"password": "caduser@123", "subdir": ["array_config_11", "array_config_12"]},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "subdir": ["array_config_13", "array_config_14"]},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "subdir": ["array_config_3", "array_config_4"]},
}

def run_remote_compute_cluster_automation(generation_idex):
    config_mapping = {}
    for user_host, config in remote_configs.items():
        for subdir in config["subdir"]:
            config_num = int(subdir.split('_')[-1])
            config_mapping[config_num] = (user_host, subdir)

    log_dir_base = f"./gen/{generation_idex}/logs"
    csv_dir = os.path.join(log_dir_base, "csv_files")
    vitals_dir = os.path.join(log_dir_base, "vitals")

    os.makedirs(log_dir_base, exist_ok=True)
    os.makedirs(csv_dir, exist_ok=True)
    os.makedirs(vitals_dir, exist_ok=True)


    # vitals_thread = threading.Thread(target=monitor_vitals, daemon=True)
    # vitals_thread.start()

    # -------- SCP CONFIG FILES BEFORE JOB START -------- #
    for config_num in range(1, 15):
        user_host, subdir = config_mapping[config_num]
        remote_user, remote_host = user_host.split("@")
        password = remote_configs[user_host]["password"]

        local_dat = f"./gen/{generation_idex}/{subdir}.dat"
        remote_path = f"/home/{remote_user}/Documents/strassen/{subdir}/{subdir}.dat"

        if os.path.exists(local_dat):
            print(f"📤 Sending {local_dat} → {remote_user}@{remote_host}:{remote_path}")
            scp_cmd = [
                "sshpass", "-p", password,
                "scp",
                "-o", "StrictHostKeyChecking=no",
                "-o", "UserKnownHostsFile=/dev/null",
                local_dat,
                f"{remote_user}@{remote_host}:{remote_path}"
            ]
            result = subprocess.run(scp_cmd)
            if result.returncode == 0:
                print("✅ File sent successfully.")
            else:
                print("❌ SCP failed.")
        else:
            print(f"⚠️ Missing local file: {local_dat}")

    # -------- JOB LAUNCH SECTION -------- #
    processes = []

    for config_num in range(1, 15):
        user_host, subdir = config_mapping[config_num]
        remote_user, remote_host = user_host.split("@")
        remote_password = remote_configs[user_host]["password"]

        remote_path = f"/home/{remote_user}/Documents/strassen/{subdir}"
        log_file = os.path.join(log_dir_base, f"{config_num}.log")

        remote_command = f"""
            cd {remote_path} && \
            python3 modify_rtl.py && \
            csh -c "source cshrc && genus -legacy_ui -files script.tcl && exit" && \
            python3 get_metrics.py
        """

        full_command = [
            "sshpass", "-p", remote_password,
            "ssh", "-tt",
            "-o", "StrictHostKeyChecking=no",
            "-o", "UserKnownHostsFile=/dev/null",
            f"{remote_user}@{remote_host}",
            remote_command
        ]

        print(f"🚀 Launching job: Config {config_num} on {user_host} in {subdir} → {log_file}")
        start_time = time.time()

        f = open(log_file, "w")
        p = subprocess.Popen(full_command, stdout=f, stderr=subprocess.STDOUT)

        processes.append({
            "process": p,
            "stdout": f,
            "config_num": config_num,
            "user_host": user_host,
            "subdir": subdir,
            "log_file": log_file,
            "start_time": start_time
        })

    # -------- WAIT & SCP METRICS SECTION -------- #
    for job in processes:
        proc = job["process"]
        proc.wait()
        end_time = time.time()

        duration = timedelta(seconds=(end_time - job["start_time"]))
        job["stdout"].write(f"\n\n✅ Time taken: {duration}\n")
        job["stdout"].close()

        print(f"✅ Completed: Config {job['config_num']} on {job['user_host']} [{job['subdir']}]")
        print(f"🕒 Time taken: {duration}")
        print(f"📄 Log saved: {job['log_file']}")

        remote_user, remote_host = job["user_host"].split("@")
        remote_password = remote_configs[job["user_host"]]["password"]
        remote_csv_path = f"/home/{remote_user}/Documents/strassen/{job['subdir']}/metrics.csv"
        local_csv = os.path.join(csv_dir, f"{job['config_num']}.csv")

        print(f"📥 Fetching metrics.csv from Config {job['config_num']} → {local_csv}")

        scp_cmd = [
            "sshpass", "-p", remote_password,
            "scp",
            "-o", "StrictHostKeyChecking=no",
            "-o", "UserKnownHostsFile=/dev/null",
            f"{remote_user}@{remote_host}:{remote_csv_path}",
            local_csv
        ]
        result = subprocess.run(scp_cmd)

        if result.returncode == 0:
            print(f"✅ Metrics saved to: {local_csv}")
        else:
            print(f"❌ Failed to fetch metrics for Config {job['config_num']}")


run_remote_compute_cluster_automation(1)

📤 Sending ./gen/1/array_config_1.dat → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_1/array_config_1.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_2.dat → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_2/array_config_2.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_3.dat → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_3/array_config_3.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_4.dat → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_4/array_config_4.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_5.dat → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_5/array_config_5.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_6.dat → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_6/array_config_6.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_7.dat → cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_config_7/array_config_7.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_8.dat → cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_config_8/array_config_8.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_9.dat → cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_config_9/array_config_9.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_10.dat → cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_config_10/array_config_10.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_11.dat → cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_config_11/array_config_11.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_12.dat → cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_config_12/array_config_12.dat
✅ File sent successfully.
📤 Sending ./gen/1/array_config_13.dat → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_13/array_config_13.dat


✅ File sent successfully.
📤 Sending ./gen/1/array_config_14.dat → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_14/array_config_14.dat
✅ File sent successfully.
🚀 Launching job: Config 1 on cadencea12@172.16.121.82 in array_config_1 → ./gen/1/logs/1.log
🚀 Launching job: Config 2 on cadencea12@172.16.121.82 in array_config_2 → ./gen/1/logs/2.log
🚀 Launching job: Config 3 on imt2022556_nishit@172.16.121.37 in array_config_3 → ./gen/1/logs/3.log
🚀 Launching job: Config 4 on imt2022556_nishit@172.16.121.37 in array_config_4 → ./gen/1/logs/4.log
🚀 Launching job: Config 5 on cadencea1@172.16.121.72 in array_config_5 → ./gen/1/logs/5.log
🚀 Launching job: Config 6 on cadencea1@172.16.121.72 in array_config_6 → ./gen/1/logs/6.log
🚀 Launching job: Config 7 on cadencea8@172.16.121.78 in array_config_7 → ./gen/1/logs/7.log
🚀 Launching job: Config 8 on cadencea8@172.16.121.78 in array_config_8 → ./gen/1/logs/8.log
🚀 Launching job: Config 9 on cadencea10@172.16.121.80 in 

✅ Completed: Config 1 on cadencea12@172.16.121.82 [array_config_1]
🕒 Time taken: 0:00:19.777058
📄 Log saved: ./gen/1/logs/1.log
📥 Fetching metrics.csv from Config 1 → ./gen/1/logs/csv_files/1.csv
✅ Metrics saved to: ./gen/1/logs/csv_files/1.csv


KeyboardInterrupt: 

# SSIM PSNR Remotes Automation

In [ ]:
remote_configs = {
    # lagging system ( needs to push along using ipc )
    "cadencea19@172.16.121.14": {"password": "caduser@123", "subdir": ['array_config_1', 'array_config_2']},
    "cadencea22@172.16.121.25": {"password": "caduser@123", "subdir": ['array_config_3', 'array_config_4']},
    "cadencea6@172.16.121.66": {"password": "caduser@123", "subdir": ['array_config_5', 'array_config_6']},
    "cadencea24@172.16.121.30": {"password": "caduser@123", "subdir": ['array_config_7', 'array_config_8']},
    "cadencea16@172.16.121.75": {"password": "caduser@123", "subdir": ['array_config_9', 'array_config_10']},
    "cadencea2@172.16.121.8": {"password": "caduser@123", "subdir": ['array_config_11', 'array_config_12']},
    "cadencea18@172.16.121.11": {"password": "caduser@123", "subdir": ['array_config_13', 'array_config_14']},
}



